In [11]:
import hashlib
import psycopg2
from psycopg2 import sql
from getpass import getpass

## Parámetros de conexión a PostgreSQL

In [12]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'dbname': 'etica',
    'user': 'postgres',
    'password': '1234'
}

## Función para hashear contraseñas con SHA-256

In [13]:
def hash_password(password: str) -> str:
    """
    Recibe una contraseña en texto plano y retorna su hash SHA-256 en hexadecimal.
    """
    return hashlib.sha256(password.encode('utf-8')).hexdigest()


## Función para generar registro_id automático según rol

In [14]:
def generar_registro_id(role: str) -> str:
    """
    Genera un registro_id con la primera letra del rol (E, P, S, A)
    seguido de 8 dígitos secuenciales basados en el mayor existente.
    """
    # Mapeo de roles a prefijos
    role_map = {
        'estudiante': 'E',
        'profesor': 'P',
        'supervisor': 'S',
        'administrador': 'A'
    }
    prefijo = role_map.get(role.lower())
    if not prefijo:
        raise ValueError(f"Rol inválido: {role}")

    conn = psycopg2.connect(**DB_CONFIG)
    try:
        cur = conn.cursor()
        # Buscar el máximo registro_id existente para este prefijo
        cur.execute(
            sql.SQL("SELECT MAX(registro_id) FROM users WHERE registro_id LIKE %s"),
            (prefijo + '%',)
        )
        max_id = cur.fetchone()[0]
        if max_id:
            secuencia = int(max_id[1:]) + 1
        else:
            secuencia = 1
        # Formatear con cero-padding a 8 dígitos
        nuevo_id = prefijo + str(secuencia).zfill(8)
        return nuevo_id
    finally:
        cur.close()
        conn.close()

## Función para insertar un usuario en la tabla `users`

In [15]:
def agregar_usuario(
    registro_id: str,
    first_name: str,
    middle_name: str,
    last_name: str,
    second_last_name: str,
    email: str,
    plain_password: str,
    role: str
):
    """
    Inserta un nuevo usuario en la tabla `users`.
    """
    pwd_hash = hash_password(plain_password)
    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        insert_query = sql.SQL(
            """
            INSERT INTO users (
                registro_id, first_name, middle_name,
                last_name, second_last_name, email,
                password, role
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """
        )
        cur.execute(
            insert_query,
            (
                registro_id,
                first_name,
                middle_name,
                last_name,
                second_last_name,
                email,
                pwd_hash,
                role.lower()
            )
        )
        conn.commit()
        print(f"✔ Usuario {email} agregado correctamente con ID {registro_id}.")

    except Exception as e:
        if conn:
            conn.rollback()
        print(f"❌ Error al agregar usuario: {e}")

    finally:
        if conn:
            cur.close()
            conn.close()


## Interfaz interactiva (ejecución desde consola)

In [16]:
if __name__ == "__main__":
    print("=== Agregar nuevo usuario ===")
    role = input("Rol (estudiante/profesor/supervisor/administrador): ")
    try:
        registro_id = generar_registro_id(role)
    except ValueError as ve:
        print(ve)
        exit(1)
    print(f"Registro ID generado: {registro_id}")

    first_name = input("Nombres: ")
    middle_name = input("Segundo nombre (opcional): ")
    last_name = input("Apellido paterno: ")
    second_last_name = input("Apellido materno: ")
    email = input("Correo electrónico: ")
    plain_password = getpass("Contraseña: ")

    agregar_usuario(
        registro_id,
        first_name,
        middle_name,
        last_name,
        second_last_name,
        email,
        plain_password,
        role
    )


=== Agregar nuevo usuario ===
Registro ID generado: P00000001
✔ Usuario a@nulllllll.com agregado correctamente con ID P00000001.
